In [2]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 14.1 MB/s eta 0:00:00


In [3]:
import json
import pandas as pd
import emoji
import unicodedata
import re
import numpy as np
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

In [4]:
model_name = "czearing/article-title-generator"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
headline_generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
def preprocess_description(event_description: str) -> str:
    event_description = emoji.replace_emoji(event_description, replace='')
    event_description = unicodedata.normalize('NFKC', event_description)
    event_description = re.sub(r'<[^>]+>', '', event_description)
    event_description = re.sub(r'https?://\S+|www\.\S+', '', event_description)
    event_description = re.sub(r'[^\w\s.,!?;:()"\'-]', '', event_description)
    event_description = re.sub(r'\s+', ' ', event_description).strip()
    return event_description

In [6]:
df = pd.read_csv('df.csv').drop(['Unnamed: 0'], axis=1)

In [7]:
p_df = df.dropna(subset=['tags'])

In [8]:
p_df = p_df.reset_index().drop('index', axis = 1)

In [12]:
p_df['text'].isna().sum()

np.int64(0)

In [13]:
p_df

,id,photo,text,tags
0,7,photos/photo_1@15-11-2019_14-17-29.jpg,📣Hi there! Want any of these?\n\nStudent Affai...,"['Volunteering', 'Job Fair']"
1,8,photos/photo_2@19-11-2019_17-20-16.jpg,Bonjour! Ça va?\nС'est la vie.. \nCroissant.\n...,['Language Learning']
2,9,photos/photo_3@21-11-2019_12-23-56.jpg,"📣On December 14, comedian Vladimir Marconi arr...","['Internship', 'Job Fair']"
3,15,photos/photo_9@27-11-2019_16-42-47.jpg,‼️Want to upgrade your programming skills? We ...,"['Programming', 'Computer Science']"
4,17,photos/photo_10@02-12-2019_12-46-45.jpg,"📣We invite you to the Soft Skills School, whic...",['Business']
...,...,...,...,...
1695,3307,photos/photo_2598@02-07-2025_11-59-57.jpg,\n\nCrazy slippers meets formal elegance — all...,['Design']
1696,3315,photos/photo_2603@03-07-2025_16-07-49.jpg,\n\nTwo great master classes are waiting for y...,['Master Class']
1697,3316,photos/photo_2604@05-07-2025_12-01-46.jpg,"🎮 ! \n\nThe speaker will be Alexey Pershanin, ...","['Lecture', 'Talk']"
1698,3320,photos/photo_2606@07-07-2025_16-13-38.jpg,"📣\n\n""Твой Ход"", together with the Ministry of...",['Talk']


In [ ]:
titles = []

for idx, row in p_df.iterrows():
    text = preprocess_description(row['text'])
    result = headline_generator(
        text,
        max_new_tokens=15,
        num_beams=10,
        do_sample=False,
        early_stopping=True
    )
    titles.append(result[0]['generated_text'])
    print(f'{len(titles) / 17:.2f}%')

0.06%
0.12%
0.18%
0.24%
0.29%
0.35%
0.41%
0.47%
0.53%
0.59%
0.65%
0.71%
0.76%
0.82%
0.88%
0.94%
1.00%
1.06%
1.12%
1.18%
1.24%
1.29%
1.35%
1.41%
1.47%
1.53%
1.59%
1.65%
1.71%
1.76%
1.82%
1.88%
1.94%
2.00%
2.06%
2.12%
2.18%
2.24%
2.29%
2.35%
2.41%
2.47%
2.53%
2.59%
2.65%
2.71%
2.76%
2.82%
2.88%
2.94%
3.00%
3.06%
3.12%
3.18%
3.24%
3.29%
3.35%
3.41%
3.47%
3.53%
3.59%
3.65%
3.71%
3.76%
3.82%
3.88%
3.94%
4.00%
4.06%
4.12%
4.18%
4.24%
4.29%
4.35%
4.41%
4.47%
4.53%
4.59%
4.65%
4.71%
4.76%
4.82%
4.88%
4.94%
5.00%
5.06%
5.12%
5.18%
5.24%
5.29%
5.35%
5.41%
5.47%
5.53%
5.59%
5.65%
5.71%
5.76%
5.82%
5.88%
5.94%
6.00%
6.06%
6.12%
6.18%
6.24%
6.29%
6.35%
6.41%
6.47%
6.53%
6.59%
6.65%
6.71%
6.76%
6.82%
6.88%
6.94%
7.00%
7.06%
7.12%
7.18%
7.24%
7.29%
7.35%
7.41%
7.47%
7.53%
7.59%
7.65%
7.71%
7.76%
7.82%
7.88%
7.94%
8.00%
8.06%
8.12%
8.18%
8.24%
8.29%
8.35%
8.41%
8.47%
8.53%
8.59%
8.65%
8.71%
8.76%
8.82%
8.88%
8.94%
9.00%
9.06%
9.12%
9.18%
9.24%
9.29%
9.35%
9.41%
9.47%
9.53%
9.59%
9.65%
9.71%
9.76%
9.82

Token indices sequence length is longer than the specified maximum sequence length for this model (1056 > 512). Running this sequence through the model will result in indexing errors


77.53%
77.59%
77.65%
77.71%
77.76%
77.82%
77.88%
77.94%
78.00%
78.06%
78.12%
78.18%
78.24%
78.29%
78.35%
78.41%
78.47%
78.53%
78.59%
78.65%
78.71%
78.76%
78.82%
78.88%
78.94%
79.00%
79.06%
79.12%
79.18%
79.24%
79.29%
79.35%
79.41%
79.47%
79.53%
79.59%
79.65%
79.71%
79.76%
79.82%
79.88%
79.94%
80.00%
80.06%
80.12%
80.18%
80.24%
80.29%
80.35%
80.41%
80.47%
80.53%
80.59%
80.65%
80.71%
80.76%
80.82%
80.88%
80.94%
81.00%
81.06%
81.12%
81.18%
81.24%
81.29%
81.35%
81.41%
81.47%
81.53%
81.59%
81.65%
81.71%
81.76%
81.82%
81.88%
81.94%
82.00%
82.06%
82.12%
82.18%
82.24%
82.29%
82.35%
82.41%
82.47%
82.53%
82.59%
82.65%
82.71%
82.76%
82.82%
82.88%
82.94%
83.00%
83.06%
83.12%
83.18%
83.24%
83.29%
83.35%
83.41%
83.47%
83.53%
83.59%
83.65%
83.71%
83.76%
83.82%
83.88%
83.94%
84.00%
84.06%
84.12%
84.18%
84.24%
84.29%
84.35%
84.41%
84.47%
84.53%
84.59%
84.65%
84.71%
84.76%
84.82%
84.88%
84.94%
85.00%
85.06%
85.12%
85.18%
85.24%
85.29%
85.35%
85.41%
85.47%
85.53%
85.59%
85.65%
85.71%
85.76%
85.82%
85.88%

In [ ]:
p_df['titles'] = titles

In [ ]:
p_df.to_csv('p_df.csv')